In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

with open('C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\text_train','r') as f:
	train=f.read().splitlines()
with open('C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\text_test','r') as f:
	test=f.read().splitlines()
X_train_text,Y_train,X_test_text,Y_test=[],[],[],[]
for doc in train:
	feature=doc.split('<<>>')
	X_train_text.append(feature[-1])
	Y_train.append(feature[0])
for doc in test:
	feature=doc.split('<<>>')
	X_test_text.append(feature[-1])
	Y_test.append(feature[0])

vectorsizer = TfidfVectorizer(min_df=4)
X_train=vectorsizer.fit_transform(X_train_text)
X_test=vectorsizer.transform(X_test_text)
model=LogisticRegression(solver='lbfgs',multi_class="multinomial")
model.fit(X_train,Y_train)
print('Pre train :',model.score(X_train,Y_train))
print('Pre test :',model.score(X_test,Y_test))

Pre train : 0.969683577868128
Pre test : 0.8287307488050982


In [ ]:
import numpy as np
import random
import time
class logistic_regression:
    def normalize(self,X):
      m=X.shape[0]
      x_ones=np.ones((m,1))
      return np.hstack((x_ones,X))

    def tranf_label(self,Y,label):
        Y_new=np.zeros(len(Y),dtype=np.int64)
        for i in range(len(Y)):
            if(str(Y[i])==str(label)):
                Y_new[i]=1
        return Y_new

    def sigmoid(self,x):
        return 1./(1+np.exp(-x))
    
    def check_stop(self,X,Y,w):
        m=X.shape[0]
        l1_grad=[]
        for i in range(m):
            l1_grad.append(sum(np.abs(self.grad(X[i],Y[i],w))))
        return np.max(l1_grad)

    def grad(self,xi,yi,w):
        return (self.sigmoid(xi.dot(w))-yi)*xi
    
    def get_loss(self,X_train,Y_train,w):
        N,d=X_train.shape
        loss=0.
        for i in range(N):
            z=self.sigmoid(X_train[i].dot(w))
            loss+=Y_train[i]*np.log(z)+(1-Y_train[i])*np.log(1-z)
        return -loss/N

    def get_w_SGD(self,X_train,Y_train,learning_rate,n_epoches):
        n_train,d=X_train.shape
        w_init=np.random.randn(d)
        w=w_old=w_init
        for it in range(n_epoches):
            rd=random.sample(range(n_train),n_train)
            for i in range(n_train):
                xi=X_train[rd[i]]
                yi=Y_train[rd[i]]
                w=w-learning_rate*self.grad(xi,yi,w)
                if(i%100==0):
                    if(np.linalg.norm(w-w_old)*1e15<1e-15):
                      return w
                w_old=w
        return w

    def fit(self,X_train,Y_train):
        X_train=np.array(X_train)
        X_train=self.normalize(X_train)
        Y_train=np.array(Y_train)
        n_label=len(set(Y_train))
        list_w=[]
        w_lr=[]
        lrates=np.arange(0.1,2,0.1)
        for lr in lrates:
          w_lr=[]
          for label in range(n_label):
            Y_new=self.tranf_label(Y_train,label)
            w=self.get_w_SGD(X_train,Y_new,lr,50)
            w_lr.append(w)
          list_w.append(w_lr)
        self.list_w=list_w
    
    def predict(self,X_test,Y_test):
        X_test=np.array(X_test)
        X_test=self.normalize(X_test)
        n_test,d=X_test.shape
        pre=[]
        pre_cnt=[]
        n_lr=len(self.list_w)
        for lr in self.list_w:
          pre.clear()
          for i in range(n_test):
              pre_xi=[]
              for w in lr:
                  pre_xi.append(self.sigmoid(X_test[i].dot(w)))
              label_xi=np.argmax(pre_xi)
              pre.append(label_xi)
          cnt=0
          for j in range(n_test):
            if(str(pre[j])==str(Y_test[j])):
              cnt+=1
          pre_cnt.append(cnt)
        return pre_cnt

def get_data(pathin):
    with open(pathin, 'r') as f:
        data = f.read().splitlines()
    text = []
    label = []
    fea=[]
    for doc in data:
        tmp = doc.split("<<>>")
        text.append(tmp[-1])
        label.append(tmp[0])
    for doc in text:
        fea_xi=[0]*20167
        tmp=doc.split()
        for id in tmp:
            x=id.split(':')
            fea_xi[int(x[0])]=float(x[1])
        fea.append(fea_xi)
    del data
    return fea,label

path_train="C:\\Users\\nql\\Desktop\\20192\\project2\\logistic regression\\data_train"
path_test="C:\\Users\\nql\\Desktop\\20192\\project2\\logistic regression\\data_test"

X_test,Y_test=get_data(path_test)
X_train,Y_train =get_data(path_train)

t=time.time()
model=logistic_regression()
model.fit(X_train,Y_train)
pre=model.predict(X_test,Y_test)
print(pre)
print(time.time()-t)